In [2]:
# Initialize python libs & SQL creads
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import json
import http.client
import certifi
import ssl
import os
import timeit
import pickle
import altair as alt
import pytz
import numpy as np
tz = pytz.timezone('America/Edmonton')

In [2]:
def release_token(accessToken):
    path = '/api/ReleaseToken'
    server = 'api.nrgstream.com'
    headers = {'Authorization': f'Bearer {accessToken}'}
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('DELETE', path, None, headers)
    res = conn.getresponse()

with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)

release_token(default_pickle['accessToken'])

In [12]:
t = datetime.now(pytz.timezone('America/Edmonton')).date() - timedelta(days=8)
t2 = datetime.now(pytz.timezone('America/Edmonton')) - timedelta(days=8)
default_pickle = {
    'accessToken':'',
    'current_data':'',
    'daily_outage_dfs':[0]*7,
    'monthly_outage_dfs':[0]*7,
    'alert_dates':{
        'Biomass & Other':t, 
        'Coal':t,
        'Dual Fuel':t,
        'Energy Storage':t,
        'Hydro':t,
        'Natural Gas':t,
        'Solar':t,
        'Wind':t,
        'Intertie':t
    }
}
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)
# Add default current values to dict
with open('./current_df.pickle', 'rb') as handle:
    current_df = pickle.load(handle)
default_pickle['current_data'] = current_df
# Add default monthly values to dict
with open('./monthly_df.pickle', 'rb') as handle:
    monthly = pickle.load(handle)
for i in range(7):
    default_pickle['monthly_outage_dfs'][i] = (t2,monthly)
# Add default daily values to dict
with open('./daily_df.pickle', 'rb') as handle:
    daily = pickle.load(handle)
for i in range(7):
    default_pickle['daily_outage_dfs'][i] = (t2,daily)
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)
default_pickle

{'accessToken': 'p7aL_3xU6o-Lc7nzQUkwwmkmfbsV1Y6IJNxbKMJUwWvVWRvOsK945xNut0m6I5RQza4UJsRZL2ZxJuuXlbLOGOCQVbvhwwFLkRXzHvF9HwKFR_ZVKSgYvuZj9bDzWktuXGNyOTC9D9efOCuDKZl6wU6nYBfc4Ei-qvCn1K15AKlhLtSt_Pgfw5_dXb9PElxEbu2Cr0O2wxBCJCZMoy71Y29Pr3kyYFaEk-ZGixIDM-QffKrFhbRIdfqY5_3_TWvOszsdKTwB7kJIBg-VvVZ1fGejz_ot5u5fCKQonKXteo0P0zNekdjfaIFZzGHXg5-nLkKtWdWEnWgNy99gTa0t_btFMrI',
 'current_data': (datetime.datetime(2022, 9, 19, 20, 5, 1, 669480, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
                      timeStamp    value    fuelType
  0   2022-09-19 00:00:00-06:00  1257.00        Coal
  1   2022-09-19 00:05:00-06:00  1262.00        Coal
  2   2022-09-19 00:10:00-06:00  1261.00        Coal
  3   2022-09-19 00:15:00-06:00  1259.00        Coal
  4   2022-09-19 00:20:00-06:00  1259.00        Coal
  ..                        ...      ...         ...
  236 2022-09-19 19:40:00-06:00   681.55  Pool Price
  237 2022-09-19 19:45:00-06:00   681.55  Pool Price
  238 2022-09-19 19:50:00

In [16]:
[t for t,df in default_pickle['monthly_outage_dfs']]

[datetime.datetime(2022, 9, 4, 13, 52, 29, 897614, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
 datetime.datetime(2022, 9, 4, 13, 52, 29, 897614, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
 datetime.datetime(2022, 9, 4, 13, 52, 29, 897614, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
 datetime.datetime(2022, 9, 4, 13, 52, 29, 897614, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
 datetime.datetime(2022, 9, 4, 13, 52, 29, 897614, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
 datetime.datetime(2022, 9, 12, 14, 0, 34, 184434, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
 datetime.datetime(2022, 9, 16, 13, 14, 11, 608997, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>)]

In [6]:
default_pickle['daily_outage_dfs'][6]

(datetime.datetime(2022, 9, 19, 20, 2, 14, 202745, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
                    timeStamp  value              fuelType
 0  2022-09-19 00:00:00-06:00  543.0              Intertie
 1  2022-09-19 01:00:00-06:00    0.0              Intertie
 2  2022-09-19 02:00:00-06:00    0.0              Intertie
 3  2022-09-19 03:00:00-06:00    0.0              Intertie
 4  2022-09-19 04:00:00-06:00    0.0              Intertie
 ..                       ...    ...                   ...
 71 2022-09-21 23:00:00-06:00    0.0  3-Day Solar Forecast
 72 2022-09-22 00:00:00-06:00    0.0  3-Day Solar Forecast
 73 2022-09-22 01:00:00-06:00    0.0  3-Day Solar Forecast
 74 2022-09-22 02:00:00-06:00    0.0  3-Day Solar Forecast
 75 2022-09-22 03:00:00-06:00    0.0  3-Day Solar Forecast
 
 [28340 rows x 3 columns])

In [5]:
default_pickle['daily_outage_dfs'][6][1][default_pickle['daily_outage_dfs'][6][1]['fuelType'] != ('3-Day Solar Forecast' or '7-Day Wind Forecast')]

,timeStamp,value,fuelType
0,2022-09-19 00:00:00-06:00,543.00,Intertie
1,2022-09-19 01:00:00-06:00,0.00,Intertie
2,2022-09-19 02:00:00-06:00,0.00,Intertie
3,2022-09-19 03:00:00-06:00,0.00,Intertie
4,2022-09-19 04:00:00-06:00,0.00,Intertie
...,...,...,...
163,2022-09-25 19:00:00-06:00,240.12,7-Day Wind Forecast
164,2022-09-25 20:00:00-06:00,282.14,7-Day Wind Forecast
165,2022-09-25 21:00:00-06:00,334.81,7-Day Wind Forecast
166,2022-09-25 22:00:00-06:00,390.90,7-Day Wind Forecast


In [5]:
with open('./temp.pickle', 'rb') as handle:
    accessToken = pickle.load(handle)
accessToken


'HMgsNGXA7rzhEsJbUPmK_XqzIIbP4ZpA6Rri6yFOeKdO4jp5Ya3_7bDbf5y9-jM8O8hEi3xt-h1Eo5aiRBULKm2ca1qdEs8CsYQQPrlnLxeLn5KuvLphWuRVDwp8RNWlV8DdB_xGcgwa7AVb9_YPMRQSaSzq_9E7_Nypznbyn2Q77XYMtwFQ0IrDPGkHULODtMhkKnjS6Nk3aWRDxpRAs2QYgv0jeiIpAjdehc1vcSXMIhic6KFCF64xY3bA1G5BVPsIE3nP0E6Zzbi-NlWfloO46X-ebFqoRz11NDVdxaSwKbjMk_BSsZbjP3Q2BW3GnrJ8vGF08FPDxQFu1Q_iy6wkVEo'